Intro to Pydantic V2


Basic Model

In [2629]:
# pip install pydantic

In [2630]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int

In [2631]:
p = Person(name='John Doe', age=30)
print(p)

name='John Doe' age=30


In [2632]:
from pydantic import ValidationError

try:
    p = Person(name='John Doe', age='junk')
except ValidationError as e:
    print(e)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='junk', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


Validation Exception

In [2633]:
try:
    Person(name=100, age='junk')
except ValidationError as e:
    exception = e

In [2634]:
exception.errors()

[{'type': 'string_type',
  'loc': ('name',),
  'msg': 'Input should be a valid string',
  'input': 100,
  'url': 'https://errors.pydantic.dev/2.10/v/string_type'},
 {'type': 'int_parsing',
  'loc': ('age',),
  'msg': 'Input should be a valid integer, unable to parse string as an integer',
  'input': 'junk',
  'url': 'https://errors.pydantic.dev/2.10/v/int_parsing'}]

In [2635]:
exception.json()

'[{"type":"string_type","loc":["name"],"msg":"Input should be a valid string","input":100,"url":"https://errors.pydantic.dev/2.10/v/string_type"},{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"junk","url":"https://errors.pydantic.dev/2.10/v/int_parsing"}]'

Deserialization of Data

In [2636]:
data = {'name': 'John Doe', 'age': 30}

In [2637]:
p = Person.model_validate(data)
p

Person(name='John Doe', age=30)

In [2638]:
data_json = '{"name": "John Doe", "age": 20}'

p = Person.model_validate_json(data_json)
p

Person(name='John Doe', age=20)

Required vs Optional Fields

In [2639]:
try:
    Person(age=29)
except ValidationError as e:
    print(e)

1 validation error for Person
name
  Field required [type=missing, input_value={'age': 29}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


In [2640]:
data = {'name': 'John Doe'}

try:
    Person.model_validate(data)
except ValidationError as e:
    print(e)

1 validation error for Person
age
  Field required [type=missing, input_value={'name': 'John Doe'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


In [2641]:
# Create a default value for a field
class Person(BaseModel):
    name: str
    age: int = 30

In [2642]:
Person.model_fields

{'name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=30)}

In [2643]:
p = Person(name='John Doe')
p

Person(name='John Doe', age=30)

Nullable Fields

In [2644]:
class Person(BaseModel):
    name: str | None = None
    age: int = 30

In [2645]:
Person.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'age': FieldInfo(annotation=int, required=False, default=30)}

In [2646]:
p = Person()
p

Person(name=None, age=30)

In [2647]:
from typing import Union

class Person(BaseModel):
    name: Union[str, None] = None
    age: int = 30

In [2648]:
Person.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'age': FieldInfo(annotation=int, required=False, default=30)}

In [2649]:
from typing import Optional

class Person(BaseModel):
    name: Optional[str] = None
    age: int = 30

In [2650]:
Person.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'age': FieldInfo(annotation=int, required=False, default=30)}

List Fields

In [2651]:
class Person(BaseModel):
    name: str
    age: int
    lucky_numbers: list[int] = []

In [2652]:
Person.model_fields

{'name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True),
 'lucky_numbers': FieldInfo(annotation=list[int], required=False, default=[])}

In [2653]:
p = Person(name='John Doe', age=30, lucky_numbers=[1, '2', 3.0])
p

Person(name='John Doe', age=30, lucky_numbers=[1, 2, 3])

In [2654]:
for number in p.lucky_numbers:
    print(number, type(number))

1 <class 'int'>
2 <class 'int'>
3 <class 'int'>


Aliases and Field Class


In [2655]:
data = {
    "id": 123,
    "First Name": "John",
    "LASTNAME": "Doe",
    "age in years": 30,
}

In [2656]:
from pydantic import Field

class Person(BaseModel):
    id_: int = Field(alias='id')
    first_name: str = Field(alias='First Name')
    last_name: str = Field(alias='LASTNAME')
    age: int = Field(alias='age in years')

p = Person.model_validate(data)
p

Person(id_=123, first_name='John', last_name='Doe', age=30)

In [2657]:
# deserialize
p.model_dump()

{'id_': 123, 'first_name': 'John', 'last_name': 'Doe', 'age': 30}

In [2658]:
p.model_dump_json()

'{"id_":123,"first_name":"John","last_name":"Doe","age":30}'

In [2659]:
p.model_dump(by_alias=True)

{'id': 123, 'First Name': 'John', 'LASTNAME': 'Doe', 'age in years': 30}

In [2660]:
p.model_dump_json(by_alias=True)

'{"id":123,"First Name":"John","LASTNAME":"Doe","age in years":30}'

In [2661]:
class Person(BaseModel):
    first_name: str | None = Field(alias='First Name', default=None)
    last_name: str | None = Field(alias='LASTNAME')

In [2662]:
data = {
    "LASTNAME": "Doe",
}

p = Person.model_validate(data)
p

Person(first_name=None, last_name='Doe')

In [2663]:
try:
    Person(first_name='John')
except ValidationError as e:
    print(e)

1 validation error for Person
LASTNAME
  Field required [type=missing, input_value={'first_name': 'John'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


Model Config: Populate By Name

In [2664]:
from pydantic import ConfigDict

class Person(BaseModel):
    model_config = ConfigDict(populate_by_name=True)

    first_name: str | None = Field(alias='First Name', default=None)
    last_name: str = Field(alias='LASTNAME')

In [2665]:
p = Person(last_name='Doe')
p

Person(first_name=None, last_name='Doe')

In [2666]:
data = {
    "LASTNAME": "Doe",
    "first_name": "John",
}

p = Person.model_validate(data)
p

Person(first_name='John', last_name='Doe')

Mutable Defaults

In [2667]:
class LotteryTicket(BaseModel):
    numbers: list[int] = []

In [2668]:
t1 = LotteryTicket()
t2 = LotteryTicket()

In [2669]:
t1.numbers.extend([1, 2, 3])
t1

LotteryTicket(numbers=[1, 2, 3])

In [2670]:
t2.numbers

[]

Default Factories

In [2671]:
from datetime import datetime, timezone

class Log(BaseModel):
    timestamp: datetime = Field(default_factory= lambda: datetime.now(tz=timezone.utc))
    message: str

In [2672]:
log1 = Log(message='Hello')

log2 = Log(message='World')

In [2673]:
log1

Log(timestamp=datetime.datetime(2025, 1, 19, 0, 1, 6, 268019, tzinfo=datetime.timezone.utc), message='Hello')

In [2674]:
log2

Log(timestamp=datetime.datetime(2025, 1, 19, 0, 1, 6, 268075, tzinfo=datetime.timezone.utc), message='World')

Custom Serializers

In [2675]:
class Model(BaseModel):
    number: float

In [2676]:
m = Model(number=1.0)
m.model_dump_json()

'{"number":1.0}'

In [2677]:
m = Model(number=1/3)
m.model_dump_json()

'{"number":0.3333333333333333}'

In [2678]:
dt = datetime.now(timezone.utc)
dt.isoformat()

'2025-01-19T00:01:06.838305+00:00'

In [2679]:
class Model(BaseModel):
    dt: datetime

In [2680]:
m  =Model(dt= datetime.now(timezone.utc))
m

Model(dt=datetime.datetime(2025, 1, 19, 0, 1, 6, 860219, tzinfo=datetime.timezone.utc))

In [2681]:
m.model_dump_json()

'{"dt":"2025-01-19T00:01:06.860219Z"}'

In [2682]:
m.model_dump()

{'dt': datetime.datetime(2025, 1, 19, 0, 1, 6, 860219, tzinfo=datetime.timezone.utc)}

In [2683]:
from pydantic import field_serializer

In [2684]:
class Model(BaseModel):
    number: float
    
    @field_serializer('number')
    def serialize_number(self, v: float) -> float:
        return round(v, 2)

In [2685]:
m = Model(number=1/3)
m.model_dump()

{'number': 0.33}

In [2686]:
m.model_dump_json()

'{"number":0.33}'

In [2687]:
class Model(BaseModel):
    number: float
    dt :  datetime
    
    @field_serializer('number')
    def serialize_number(self, v: float) -> float:
        return round(v, 2)
    
    @field_serializer('dt', when_used="json-unless-none")
    def serialize_dt_to_json(self, v: datetime) -> str:
        return v.strftime('%Y-%m-%d')

In [2688]:
m = Model(number=1/3, dt=datetime.now(timezone.utc))
m.model_dump()

{'number': 0.33,
 'dt': datetime.datetime(2025, 1, 19, 0, 1, 7, 14683, tzinfo=datetime.timezone.utc)}

In [2689]:
m.model_dump_json()

'{"number":0.33,"dt":"2025-01-19"}'

Custom Validators

In [2690]:
from pydantic import field_validator

In [2691]:
class Model(BaseModel):
    number: int
    
    @field_validator('number')
    @classmethod
    def validate_number(cls, v: int) -> int:
        print(f"running custom validator for {v}, type: {type(v)}")
        return abs(v)

In [2692]:
m = Model(number=-1)
m

running custom validator for -1, type: <class 'int'>


Model(number=1)

In [2693]:
m  = Model(number="-1")
m

running custom validator for -1, type: <class 'int'>


Model(number=1)

In [2694]:
# validating data

class Model(BaseModel):
    numbers: list[int] = []

    @field_validator('numbers')
    @classmethod
    def ensure_unique_numbers(cls, v: list[int]):
        if len(v) != len(set(v)):
            raise ValueError('numbers must be unique')
        return v

In [2695]:
Model(numbers=[1, 2, 3])


Model(numbers=[1, 2, 3])

In [2696]:
try:
    Model(numbers=[1, 2, '2'])
except ValidationError as e:
    print(e)

1 validation error for Model
numbers
  Value error, numbers must be unique [type=value_error, input_value=[1, 2, '2'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


Nested Models

In [2697]:
data = {
    'firstName': 'John',
    'lastName': 'Doe',
    'age': 30,
    'born':{
        "place": {
            "city": "New York",
            "country": "USA"
        },
        "date": "1990-01-01",
    },
}

In [2698]:
from datetime import date

In [2699]:
class Place(BaseModel):
    city: str
    country: str


class Born(BaseModel):
    place: Place
    dt: date = Field(alias='date')


class Person(BaseModel):
    first_name: str = Field(alias='firstName')
    last_name: str = Field(alias='lastName')
    age: int
    born: Born

In [2700]:
john = Person.model_validate(data)
john

Person(first_name='John', last_name='Doe', age=30, born=Born(place=Place(city='New York', country='USA'), dt=datetime.date(1990, 1, 1)))

In [2701]:
john.born.dt

datetime.date(1990, 1, 1)

In [2702]:
john.model_dump_json()

'{"first_name":"John","last_name":"Doe","age":30,"born":{"place":{"city":"New York","country":"USA"},"dt":"1990-01-01"}}'

In [2703]:
from pprint import pprint

pprint(john.model_dump())

{'age': 30,
 'born': {'dt': datetime.date(1990, 1, 1),
          'place': {'city': 'New York', 'country': 'USA'}},
 'first_name': 'John',
 'last_name': 'Doe'}


In [2704]:
print(john.model_dump_json(indent=4))

{
    "first_name": "John",
    "last_name": "Doe",
    "age": 30,
    "born": {
        "place": {
            "city": "New York",
            "country": "USA"
        },
        "dt": "1990-01-01"
    }
}
